In [1]:
import pandas as pd

In [2]:
file = r"..\metadata_20241010T1031.csv"

river_name_list = ["Nelson River"]

In [3]:
df = pd.read_csv(file, encoding="utf-8-sig")
df.head()

Station Number                                       Station Name Province  \
0        01AA002    DAAQUAM (RIVIERE) EN AVAL DE LA RIVIERE SHIDGEL       QC   
1        01AD001  MADAWASKA (RIVIÈRE) À 6 KM EN AVAL DU BARRAGE ...       QC   
2        01AD002                      SAINT JOHN RIVER AT FORT KENT       NB   
3        01AD003        ST. FRANCIS RIVER AT OUTLET OF GLASIER LAKE       NB   
4        01AD004                     SAINT JOHN RIVER AT EDMUNDSTON       NB   

         Status  Latitude  Longitude  Year From  Year To  \
0  Discontinued  46.55750  -70.08111       1967     1977   
1  Discontinued  47.54833  -68.63639       1918     2005   
2        Active  47.25806  -68.59583       1926     2024   
3        Active  47.20661  -68.95694       1951     2024   
4        Active  47.36078  -68.32489       1967     2024   

   Gross Drainage Area (km2)  Effective Drainage Area (km2) Regulation  \
0                      598.0                            NaN          N   
1                     2690.0                            NaN          R   
2                    14700.0                            NaN          N   
3                     1350.0                            NaN          N   
4                    15500.0                            NaN          N   

        Data Type Operation Schedule Sediment RHBN Real-Time  \
0            Flow         Continuous        N    N         N   
1            Flow         Continuous        N    N         N   
2            Flow         Continuous        N    Y         N   
3  Flow and Level         Continuous        N    Y         Y   
4  Flow and Level         Continuous        N    N         Y   

                                Datum Name Publishing Office  \
0                               ARBITRAIRE       QUEBEC CITY   
1                               ARBITRAIRE       QUEBEC CITY   
2  INTERNATIONAL BOUNDARY COMMISSION DATUM         DARTMOUTH   
3                            ASSUMED DATUM         DARTMOUTH   
4          GEODETIC SURVEY OF CANADA DATUM         DARTMOUTH   

                         Operating Agency Contributed  
0  MINISTERE DE L'ENVIRONNEMENT DU QUEBEC           Y  
1  MINISTERE DE L'ENVIRONNEMENT DU QUEBEC           Y  
2         UNITED STATES GEOLOGICAL SURVEY           Y  
3                                     NaN           N  
4                                     NaN           N

In [4]:
def modify_river_name(river):
    river_dict = {
        "Mackenzie River": "Mackenzie",
        "Coppermine River": "Coppermine",
        "Big River": "Big",
        "Tree River": "Tree",
        "Burnside River": "Burnside",
        "Gordon River": "Gordon",
        "Ellice River": "Ellice",
        "Back River": "Back",
        "Kazan River": "Kazan",
    }
    return river_dict.get(river, river)


def modify_station_name(station):
    station_dict = {
        "7.5 Km Down Of Buur'S Mouth": "7.5 km down of Buur's mouth",
    }
    return station_dict.get(station, station)


def extract_river_station(river_station):
    keywords = [" Near ", " Nr ", "Above", "Below", " River ", " - "]
    keywords2 = [" Near ", " Nr ", "Above", "Below"]
    river_station = river_station.title()

    for river_name in river_name_list:
        if river_station.startswith(river_name):
            # rest of the string is station name
            return river_name, river_station.replace(river_name, "").strip()

    if " At " in river_station:
        river_station = river_station.split(" At ", maxsplit=1)
        assert len(river_station) == 2
        river = river_station[0]
        station = river_station[1]
        return river.strip(), station.strip()

    for keyword in keywords:
        if keyword in river_station:
            river_station = river_station.split(keyword, maxsplit=1)
            assert len(river_station) == 2
            river = river_station[0]
            station = river_station[1]
            if keyword in keywords2:
                station = keyword + station
            return river.strip(), station.strip()

    return river_station, ""

In [5]:
df[["River", "Station"]] = df.apply(
    lambda x: extract_river_station(x["Station Name"]), axis=1, result_type="expand"
)

df["River"] = df["River"].apply(modify_river_name)
df["Station"] = df["Station"].apply(modify_station_name)

df.rename(columns={"Latitude": "Lat", "Longitude": "Lon"}, inplace=True)

In [6]:
df

Station Number                                       Station Name  \
0           01AA002    DAAQUAM (RIVIERE) EN AVAL DE LA RIVIERE SHIDGEL   
1           01AD001  MADAWASKA (RIVIÈRE) À 6 KM EN AVAL DU BARRAGE ...   
2           01AD002                      SAINT JOHN RIVER AT FORT KENT   
3           01AD003        ST. FRANCIS RIVER AT OUTLET OF GLASIER LAKE   
4           01AD004                     SAINT JOHN RIVER AT EDMUNDSTON   
...             ...                                                ...   
6834        11AE015                         GIRARD CREEK NEAR CORONACH   
6835        11AE018                   POPLAR RIVER ABOVE BOLSTER CREEK   
6836        11AF002          BIG MUDDY CREEK AT INTERNATIONAL BOUNDARY   
6837        11AF004           BEAVER CREEK NEAR INTERNATIONAL BOUNDARY   
6838        11AF005             BEAVER CREEK AT INTERNATIONAL BOUNDARY   

     Province        Status       Lat        Lon  Year From  Year To  \
0          QC  Discontinued  46.55750  -70.08111       1967     1977   
1          QC  Discontinued  47.54833  -68.63639       1918     2005   
2          NB        Active  47.25806  -68.59583       1926     2024   
3          NB        Active  47.20661  -68.95694       1951     2024   
4          NB        Active  47.36078  -68.32489       1967     2024   
...       ...           ...       ...        ...        ...      ...   
6834       SK        Active  49.11250 -105.56444       1980     2024   
6835       SK  Discontinued  49.10333 -106.02000       1980     1980   
6836       SK  Discontinued  49.00833 -104.86111       1977     1977   
6837       AB  Discontinued  48.98889 -105.01111       1949     1952   
6838       SK  Discontinued  49.00000 -105.03500       1982     1995   

      Gross Drainage Area (km2)  Effective Drainage Area (km2)  ...  \
0                         598.0                            NaN  ...   
1                        2690.0                            NaN  ...   
2                       14700.0                            NaN  ...   
3                        1350.0                            NaN  ...   
4                       15500.0                            NaN  ...   
...                         ...                            ...  ...   
6834                      880.0                          269.0  ...   
6835                      308.0                            NaN  ...   
6836                       88.6                           85.2  ...   
6837                      458.0                          448.0  ...   
6838                      387.0                          375.0  ...   

     Operation Schedule Sediment RHBN Real-Time  \
0            Continuous        N    N         N   
1            Continuous        N    N         N   
2            Continuous        N    Y         N   
3            Continuous        N    Y         Y   
4            Continuous        N    N         Y   
...                 ...      ...  ...       ...   
6834           Seasonal        N    N         N   
6835      Miscellaneous        N    N         N   
6836      Miscellaneous        N    N         N   
6837           Seasonal        N    N         N   
6838           Seasonal        N    N         N   

                                   Datum Name Publishing Office  \
0                                  ARBITRAIRE       QUEBEC CITY   
1                                  ARBITRAIRE       QUEBEC CITY   
2     INTERNATIONAL BOUNDARY COMMISSION DATUM         DARTMOUTH   
3                               ASSUMED DATUM         DARTMOUTH   
4             GEODETIC SURVEY OF CANADA DATUM         DARTMOUTH   
...                                       ...               ...   
6834                            ASSUMED DATUM            REGINA   
6835                                      NaN            REGINA   
6836                                      NaN            REGINA   
6837                                      NaN            REGINA   
6838                            ASSUMED DATUM   

In [7]:
df.to_csv(r"..\Station Info.csv", index=False, encoding="utf-8-sig")